In [1]:
parent_folder = '/Users/ravi.tej/Desktop/ML/Recommendations/hubble/'
from hydra import compose, initialize
import os

import xml.etree.ElementTree as ET

tree = ET.parse('../../conf/application.run.xml')
root = tree.getroot()

envs_element = root.find('./configuration/envs')
for variable in envs_element.findall('env'):
    name = variable.get('name')
    value = variable.get('value')
    os.environ[name] = value

import sys
sys.path.append('/Users/ravi.tej/Desktop/ML/Recommendations/hubble/')

In [2]:
from src.HybridSearchService import HybridSearchService
from src.QueryService import QueryService
from sql.PostgresDatabaseOperation import PostgresDatabaseOperation
from datetime import datetime

/Users/ravi.tej/anaconda3/envs/recoenv/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:671: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
from src.SummaryService import SummaryService

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from src.ArticleService import ArticleService

In [5]:
from transformers import AutoTokenizer

In [6]:
model_id = "teknium/OpenHermes-2.5-Mistral-7B"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=True, use_fast=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
system_prompt = '''You're the lead editor for a financial and business website. You answer user queries by using relevant articles that you've found by strictly following the below rules:
*valid_queries: questions about finance, business, or relevant entities like India, RBI, Reliance, Fed etc
*invalid_queries: non financial or business questions or inappropriate content
response_instructions:
1.query is valid and answerable with the articles you found - summarize the information clearly and concisely with a headline and followed by 3 to 6 points with following guidelines: 
Prefix the point with a relevant label one emoji for that point. The points should overall capture the essence of a good response for the user query, but at the same time as brief as possible - a few words to a maximum of 1 sentence per point. Start with the most important point for the query. The labels should be human readable avoid use of _ or programming symbols. Headline should be between 5 to 15 words and tailored to the query and the points you are going to generate
Use the below markdown format called ‘response_markdown_format’: “**Headline** <br> <br> emoji1 **label1**: point1 <br><br> **emoji2 label2**: point2 <br><br>…”
2.query is valid but the exact answer isn't in the articles - mention that you are not able to find the exact answer for this, but found the related information. Respond the related information using response_markdown_format
3.query is valid but there's no relevant information - apologize and say you can't find the answer
4.query is invalid - politely decline noting your focus on finance and business
General Response Guidelines:
*Don't provide information not explicitly found in the articles
*If the article is older than what the user expects from query, highlight the period of the information. If articles are very recent or of today, then no need to specify it
*The website is visited by people all age groups. Avoid content or emojis which are unprofessional
*If the answers provide investment advice, at the end advice without the emoji and label to the users have to do additional though research and to consult an expert. Do not follow the format of response_markdown_format for this
*Be brief. no preamble or postamble. directly generate the responses according to response_instructions
The most relevant articles are extracted in the following json format: {article_0: {‘title’: ‘’, ‘published_date’: ‘’, ‘content’: ‘’} || ...}. User’s query is provided as user_query'''

In [8]:
import json

In [9]:
def convert_dates_to_readable_format(dt):
    # Sample datetime string in the format '%Y-%m-%dT%H:%M:%S'
    datetime_obj = datetime.strptime(dt, '%Y-%m-%dT%H:%M:%S%z')
    return datetime_obj.strftime('%d %B %Y %H:%M')

In [10]:
def create_reference_articles(top_article_ids):
    articles = ArticleService.get_Articles_from_list(top_article_ids)
    
    arts = []
    for article in list(articles.values()):
        cur_art = article
        truncated_text = SummaryService.truncate_text_to_token_limit(article.cleaned_text, token_limit=896)
        cur_art.truncated_text = truncated_text.strip()
        cur_art.formatted_date = convert_dates_to_readable_format(article.published_time)
        arts.append(cur_art)
    return arts

In [11]:
def generate_formatted_reference_string(reference_articles):
    references = ''
    ref_count = 0
    for ref in reference_articles:
        cur_json = f" ||article_{ref_count}: {{'title': {ref.title}, 'published_date': {ref.formatted_date}, 'content': {ref.truncated_text}}}"
        ref_count += 1
        references += cur_json
    return references

In [ ]:
# def generate_formatted_reference_string(reference_articles):
#     references = ''
#     ref_count = 0
#     for ref in reference_articles:
#         cur_json = f" ||article_{ref}: {{'title': '', 'published_date': '', 'content': ''}}"
#         ref_count += 1
#         references += cur_json
#     return references

In [14]:
def create_prompt_for_query_references(search_query, top_hits, today_date=None):
    if today_date:
        today_date_string = f" ||today date: {convert_dates_to_readable_format(today_date)}"
    else:
        today_date_string = f" ||today date: {datetime.now().strftime('%d %B %Y')}"
    query_string = f" ||user_query: {search_query}"
    # reference_articles = create_reference_articles(top_hits)
    references_string = generate_formatted_reference_string(range(3))
    user_prompt = today_date_string + query_string + references_string
    return system_prompt + user_prompt

In [51]:
top_hits = [x['article_id'] for x in a['searchArticleIds']][:3]

In [56]:
# s = create_reference_articles(top_hits)

In [61]:
create_prompt_for_query_references('what is the longest value of things which happen today', None)

"You're the lead editor for financial and business website. You answer user queries by using relevant articles that you've found by strictly following the below rules:\n*valid_queries: questions about finance, business, or relevant entities like India, RBI, Reliance, Fed etc\n*invalid_queries: non financial or business questions or inappropriate content\nresponse_instructions:\n1.query is valid and answerable with the articles you found - summarize the information clearly and concisely with a headline and followed by 3 to 6 points with following guidelines: \nPrefix the point with a relevant label one emoji for that point. The points should overall capture the essence of a good response for the user query, but at the same time as brief as possible - a few words to a maximum of 1 sentence per point. Start with the most important point for the query. The labels should be human readable avoid use of _ or programming symbols. Headline should be brief but tailored to the query and the point

In [15]:
total_template_tokens = len(tokenizer.encode(create_prompt_for_query_references('', None)))

AttributeError: 'int' object has no attribute 'title'

In [90]:
total_template_tokens

648

In [91]:
len(tokenizer.encode('OneSpan to Announce Fourth Quarter and Fiscal Year 2023 Financial Results on March 6, 2024'))

30

### Calculting the token limit for title

In [92]:
total_template_tokens = 648
date_tokens = len(tokenizer.encode('12 February 2024'))
article_content_tokens = 896
search_query_tokens = 48
output_limit = 512
total_limit = 4096
num_articles = 3
buffer = 24
title_limit = (total_limit - (total_template_tokens + buffer + search_query_tokens + (date_tokens + article_content_tokens) * num_articles + output_limit))/num_articles

In [93]:
title_limit

48.666666666666664

In [ ]:
title_limit = 

In [10]:
# s = create_reference_articles(top_hits)

# p = create_prompt_for_query_references(query, top_hits, today_date='2024-02-05T23:59:59+05:30')

In [76]:
4096-368

3728

In [77]:
3728-683

3045

In [78]:
3045/3

1015.0

In [ ]:
non_prompt_tokens = 683 - 512

#### Queries from DB

In [3]:
with PostgresDatabaseOperation() as cursor:
    sql = 'SELECT DISTINCT query_text FROM search_query_embeddings'
    cursor.execute(sql)
    results = cursor.fetchall()

In [4]:
insight_queries = [x[0].lower() for x in results]

#### Queries from GPT

In [5]:
abstract = '''real estate investment in Mumbai;;upcoming financial technology trends;;latest fixed deposit interest rates in india;;hdfc bank quarterly earnings;;impact of monsoon on indian agriculture stocks;;cryptocurrency regulations in india;;comparative analysis of health insurance plans;;state of biotechnology stocks;;gold vs real estate investment;;tips for first-time stock investors in india;;renewable energy sector growth in india;;nifty midcap stocks to watch;;mutual funds for retirement savings;;how to start a small business in india;;indian economy post-covid recovery;;upcoming government policies on digital payments;;analysis of indian automobile sector;;benefits of robo-advisors in investing;;latest updates in indian pharmaceutical industry;;long-term impact of gst on indian economy;;sbi life insurance stock analysis;;best savings account interest rates in india;;upcoming technology ipos in india;;analysis of india's textile industry;;effects of us-china trade war on india;;artificial intelligence startups in india;;best education loan providers in india;;stock market basics for beginners;;impact of brexit on indian economy;;latest news on indian real estate market;;indian startups to watch in 2024;;advantages of national pension scheme;;how to diversify investments in india;;indian renewable energy policy updates;;best mutual funds for tax saving in india;;current gold rate trends in india;;review of the latest union budget;;tips for managing home loans effectively;;indian fintech landscape post-pandemic;;analysis of the indian hospitality industry;;ways to invest in international stock markets from india;;current trends in indian bond market;;latest news on rbi's monetary policy;;how to plan for early retirement in india;;comparison of online stock trading platforms in india;;implications of electric vehicles on indian economy;;future of blockchain in indian banking;;analysis of indian consumer electronics market;;digital marketing trends in india;;indian aviation industry outlook;;best tax-saving strategies for high-income earners in india;;start-up financing options in india;;recent trends in indian e-wallet usage;;new government schemes for small businesses in india;;analysis of indian agriculture sector;;corporate social responsibility in indian companies;;current trends in indian luxury goods market;;indian space industry investments;;best practices for financial risk management;;upcoming infrastructure projects in india;;market analysis of indian spices;;investment opportunities in indian education sector;;recent trends in indian online retail;;current state of indian healthcare industry;;latest developments in indian solar energy sector;;digital transformation in indian banking;;growth of e-commerce in india post-covid;;how to analyze stock market trends;;indian manufacturing sector outlook;;comparison of term vs whole life insurance;;indian rupee forecast against us dollar;;how to save on taxes with mutual funds in india;;latest trends in indian IT outsourcing;;impact of artificial intelligence on indian jobs;;current status of indian steel industry;;tips for investing in indian real estate;;recent changes in indian labor laws;;analysis of indian dairy industry;;how to start investing in mutual funds in india;;personal finance management for young adults in india;;latest trends in indian organic food market;;state of venture capital in india;;global economic trends affecting india;;how to build an emergency fund in india;;investment in indian electric vehicle market;;latest news on indian railways privatization;;how to secure a mortgage in india;;impact of global warming on indian economy;;latest developments in indian satellite technology;;best dividend-paying stocks in india;;upcoming regulatory changes in indian banking;;analysis of indian petrochemical industry;;how to plan for children's education in india;;growth prospects of indian IT sector;;social impact investing in india;;current trends in indian rural economy;;how to start a franchise business in india;;indian online gaming industry analysis;;long-term investment strategies in india;;recent developments in indian renewable energy policies;;how to manage debt effectively in india;;current state of indian fisheries industry;;tips for successful forex trading in india;;analysis of indian cosmetic industry;;impact of digital currency on indian economy;;current trends in indian luxury real estate;;ways to invest in indian startups;;best small-cap stocks in india;;recent trends in indian mobile payments;;how to plan a financially secure retirement in india;;analysis of the indian shipping industry;;growth of artificial intelligence in indian healthcare;;latest news on indian tax reforms;;how to invest in indian government bonds;;indian automobile sector post-covid recovery;;tips for buying gold in india;;analysis of indian food processing industry;;how to start a tech startup in india;;current trends in indian organic farming;;upcoming e-commerce trends in india;;impact of us elections on indian economy;;how to choose a financial advisor in india;;recent trends in indian solar power projects;;best practices for estate planning in india;;latest trends in indian online education;;analysis of indian textile exports;;indian stock market predictions for 2024;;how to start an import-export business in india;;impact of oil prices on indian economy;;tips for investing in indian bonds;;growth of digital banking in india;;upcoming trends in indian fintech sector;;how to manage student loans in india;;recent changes in indian foreign investment policy;;analysis of indian handicraft industry;;ways to improve credit score in india;;investment strategies for high net worth individuals in india;;current state of indian tourism industry;;how to invest in indian blue-chip stocks;;analysis of indian cold chain industry;;tips for first-time home buyers in india;;latest news on indian economic growth;;impact of climate change on indian agriculture;;how to start a food business in india;;recent trends in indian online shopping;;best long-term investments in india;;current state of indian pharmaceutical exports;;how to build a diversified investment portfolio in india;;latest trends in indian startup ecosystem;;impact of 5g technology on indian economy;;tips for saving for a home in india;;analysis of indian beverage industry;;how to start a small manufacturing business in india;;upcoming trends in indian retail sector;;how to manage a portfolio of indian stocks;;recent changes in indian banking regulations;;analysis of indian leather industry;;how to invest in indian real estate from abroad;;growth of digital payments in india post-covid;;best practices for budgeting in india;;recent trends in indian e-commerce sector;;current state of indian electric vehicle industry;;tips for investing in indian mutual funds;;analysis of indian oil and gas sector;;how to start a consulting business in india;;latest developments in indian space technology;;best small business ideas in india;;recent trends in indian digital media;;upcoming financial regulations in india;;how to build wealth in india;;analysis of indian renewable energy market;;tips for buying property in india;;recent changes in indian tax laws;;how to start a retail business in india;;current state of indian logistics industry;;how to manage personal finances during a recession in india;;latest news on indian fintech startups;;analysis of indian defense industry;;how to save for college in india;;growth of online education in india post-covid;;upcoming trends in indian healthtech sector;;how to invest in stock market for beginners in india;;recent changes in indian corporate governance;;analysis of indian food delivery industry;;ways to reduce tax liability in india;;investment opportunities in indian biotech sector;;current state of indian real estate market post-covid;;how to plan for financial emergencies in india;;latest news on indian digital currency;;analysis of indian automotive components industry;;how to start an e-commerce business in india;;recent trends in indian venture capital;;best practices for investing in indian stock market;;current state of indian aerospace industry;;tips for managing credit card debt in india;;analysis of indian gems and jewelry industry;;how to start a freelance business in india;;growth of telemedicine in india post-covid;;upcoming trends in indian edtech sector;;how to invest in mutual funds online in india;;recent changes in indian insurance regulations;;analysis of indian fast food industry;;ways to build credit history in india;;investment strategies for millennials in india;;current state of indian railway infrastructure;;how to plan a budget-friendly vacation in india;;latest news on indian e-wallet startups;;analysis of indian marine industry;;how to start a digital marketing agency in india;;recent trends in indian artificial intelligence;;best practices for saving for retirement in india;;current state of indian telecommunications industry;;tips for buying stocks in india;;analysis of indian natural gas sector;;how to start a fitness business in india;;growth of e-learning in india post-covid;;upcoming trends in indian fintech startups;;how to invest in gold in india;;recent changes in indian financial regulations;;analysis of indian organic food exports;;ways to improve financial literacy in india;;investment opportunities in indian renewable energy sector;;current state of indian coal industry;;how to manage student loan debt in india;;latest news on indian blockchain startups;;analysis of indian shipbuilding industry;;how to start an online store in india;;recent trends in indian machine learning;;best practices for managing student loans in india;;current state of indian petrochemical exports;;tips for investing in the indian stock market;;analysis of indian spices exports;;how to start a travel agency in india;;growth of fintech in india post-covid;;upcoming trends in indian agrotech sector;;how to invest in the indian stock market from abroad;;recent changes in indian startup regulations;;analysis of indian dairy exports;;ways to invest in real estate in india;;investment strategies for young professionals in india;;current state of indian infrastructure development;;how to plan a wedding on a budget in india;;latest news on indian cryptocurrency startups;;analysis of indian handicraft exports;;how to start a boutique in india;;recent trends in indian robotics;;best practices for investing in mutual funds in india;;current state of indian textile exports;;tips for managing home loan repayments in india;;analysis of indian tea exports;;how to start a food truck business in india;;growth of online retail in india post-c'''
companies = '''TCS vs Infosys: Which is better for long-term investment?;;Reliance Industries dividend history;;HDFC Bank vs ICICI Bank: best savings account options;;Adani Enterprises growth prospects in 2024;;Biocon's latest biopharmaceutical innovations;;Tata Motors electric vehicle strategy;;Larsen & Toubro infrastructure projects update;;Axis Bank stock performance analysis;;Maruti Suzuki's market share in Indian automobile industry;;Wipro's expansion in digital services;;SBI Life Insurance vs HDFC Life Insurance;;Mahindra & Mahindra's role in agribusiness;;NTPC renewable energy initiatives;;Bajaj Finance customer satisfaction ratings;;Hindalco Industries' aluminum market strategies;;HCL Technologies global outreach;;Airtel vs Jio: best telecom services;;Sun Pharma's latest drug approvals;;ITC's diversification into FMCG;;Dr. Reddy’s Laboratories R&D investments;;Kotak Mahindra Bank's digital banking growth;;ONGC's exploration projects in 2024;;Zomato vs Swiggy: market dominance;;Ashok Leyland's innovation in commercial vehicles;;Vedanta Resources' mining operations;;Nestle India's new product launches;;Britannia Industries vs Parle Products in the biscuit market;;Piramal Enterprises' pharmaceutical developments;;DLF's role in Indian real estate;;Hero MotoCorp's motorcycle technology advancements;;UltraTech Cement's market positioning;;Cipla's healthcare contributions;;Tech Mahindra's IT solutions;;Yes Bank recovery strategies;;Apollo Hospitals' healthcare services;;Patanjali Ayurved's market impact;;LIC's insurance policy offerings;;GAIL India's natural gas projects;;Tata Steel vs JSW Steel in the steel industry;;HDFC vs SBI home loan rates;;ICICI Lombard's general insurance services;;Bharat Petroleum's fuel retail strategy;;Godrej Consumer Products' sustainability initiatives;;Eicher Motors' Royal Enfield sales in India;;IndusInd Bank's financial services;;TVS Motor's new two-wheeler launches;;Coal India's environmental initiatives;;L&T Finance Holdings' investment services;;Future Retail's growth post-pandemic;;Havells India's electronic products market;;RIL's JioMart expansion plans;;Asian Paints' decorative paints segment;;PNB's banking reforms post-scam;;Bharti Airtel's 5G rollout plans;;Muthoot Finance's gold loan services;;Aditya Birla Group's conglomerate strategies;;Indian Oil Corporation's energy diversification;;Jubilant FoodWorks' Domino's Pizza operations;;Titan Company's jewelry business;;Bajaj Auto's export strategies;;Amara Raja Batteries' automotive battery innovations;;UPL's agrochemical market share;;MRF's tire manufacturing in India;;Bandhan Bank's microfinance services;;Shree Cement's production capabilities;;Balkrishna Industries' off-highway tire market;;Avenue Supermarts' D-Mart expansion;;Dabur India's herbal products;;Motherson Sumi's automotive components;;Pidilite Industries' adhesive market;;Berger Paints' decorative paint trends;;Vodafone Idea's telecom revival plans;;Jindal Steel & Power's manufacturing capabilities;;Divi's Laboratories' API production;;Grasim Industries' textile business;;Page Industries' Jockey brand in India;;Endurance Technologies' auto components;;United Breweries' beer market share;;Tata Consumer Products' tea and coffee business;;Exide Industries' battery solutions;;Escorts' agricultural machinery innovations;;Bharat Forge's forging business;;Indian Hotels Company's Taj hotel brand;;Mindtree's digital transformation services;;PVR's cinema chain growth;;Crompton Greaves' consumer electricals;;ACC's cement manufacturing;;Fortis Healthcare's hospital network;;Hindustan Zinc's mining operations;;IDFC First Bank's retail banking services;;Marico's consumer goods market;;Cummins India's power solutions;;Aurobindo Pharma's generic drug market;;Federal Bank's banking services;;Torrent Pharmaceuticals' drug development;;Max Financial Services' insurance products;;Blue Dart Express' logistics services;;JK Cement's construction materials;;Bosch India's automotive technology;;Torrent Power's electricity generation;;Jubilant Life Sciences' pharmaceuticals and life sciences;;Glenmark Pharmaceuticals' innovation in medicine;;IRCTC's rail ticketing and catering services;;Emami's personal care products;;Radico Khaitan's liquor market;;Bharat Electronics' defense electronics;;Syngene International's contract research services;;Voltas' air conditioning market;;TV18 Broadcast's media network;;Varun Beverages' PepsiCo franchise in India;;Minda Industries' auto ancillary products;;KEC International's power transmission projects;;Bata India's footwear market;;Persistent Systems' software services;;AU Small Finance Bank's banking for the unbanked;;Dish TV's DTH services;;Apollo Tyres' global tire market;;KPIT Technologies' automotive software;;GMR Infrastructure's airport development;;JSW Energy's power generation;;Zydus Wellness' consumer health products;;Sterlite Technologies' optical fiber manufacturing;;Lupin's pharmaceutical formulations;;Bharat Electronics' defense technologies;;RBL Bank's financial performance;;V-Guard Industries' electrical appliances;;Hindustan Aeronautics' aerospace manufacturing;;Biocon Biologics' biosimilar products;;Alembic Pharmaceuticals' drug research;;Thermax's energy solutions;;Welspun India's textile manufacturing;;Hindustan Copper's copper production;;Indiabulls Housing Finance's mortgage lending;;Kajaria Ceramics' tile manufacturing;;Cholamandalam Investment's financial services;;Schaeffler India's automotive and industrial bearings;;NBCC India's real estate development;;Tata Chemicals' specialty chemicals;;L&T Technology Services' engineering solutions;;Sundaram Finance's financial services;;CESC's power supply;;Adani Green Energy's renewable energy projects;;Tata Power's energy portfolio;;Phoenix Mills' retail and commercial spaces;;Kansai Nerolac Paints' coatings business;;Prestige Estates' real estate projects;;Hindustan Petroleum's oil refining and marketing;;Bajaj Finserv's financial services;;Sunteck Realty's luxury real estate;;SAIL's steel production;;Mahindra Holidays' vacation ownership;;ABB India's electrical equipment;;DLF Cyber City's commercial real estate;;Interglobe Aviation's IndiGo airline;;Ashoka Buildcon's infrastructure projects;;Nykaa's online beauty retail;;Balrampur Chini Mills' sugar manufacturing;;Astral Poly Technik's plumbing and drainage systems;;Canara Bank's banking and financial services;;Mphasis' IT services;;Kirloskar Oil Engines' diesel engines;;Gujarat Gas' natural gas distribution;;Cochin Shipyard's shipbuilding and repair;;Nazara Technologies' gaming and sports media;;Bharat Dynamics' defense equipment;;JK Tyre & Industries' tire manufacturing;;Aavas Financiers' housing finance;;HDFC AMC's asset management services;;SpiceJet's airline operations;;ICRA's credit rating services;;Relaxo Footwears' footwear products;;Bharat Rasayan's agrochemical manufacturing;;Bajaj Electricals' consumer appliances;;CreditAccess Grameen's microfinance services;;Polycab India's wires and cables;;Apollo Micro Systems' electronic systems;;VST Tillers Tractors' agricultural machinery;;Lakshmi Machine Works' textile machinery;;Sterling and Wilson Solar's solar EPC services;;Shilpa Medicare's pharmaceutical manufacturing;;Hindustan Foods' food processing;;Greenply Industries' plywood and laminates;;Navin Fluorine International's specialty fluorochemicals;;Mahindra Logistics' supply chain solutions;;Cera Sanitaryware's bathroom fittings;;Godrej Agrovet's agribusiness;;KRBL's basmati rice products;;Kalyan Jewellers' gold and diamond jewelry;;JK Paper's paper products;;Mahanagar Gas' city gas distribution;;Indian Energy Exchange's power trading platform;;PNC Infratech's infrastructure construction;;Route Mobile's cloud communication services;;Max Healthcare's hospital network;;Suprajit Engineering's automotive cables;;JMC Projects' construction and engineering;;CSB Bank's banking services;;Crompton Greaves Consumer Electricals' electrical products;;Quess Corp's business services;;Hatsun Agro Product's dairy products;;Sobha's real estate development;;Bharat Forge's forging and machining;;Balkrishna Paper Mills' paper manufacturing;;TeamLease Services' staffing solutions;;HDFC Life's insurance products;;KNR Constructions' infrastructure projects;;ICICI Prudential Life Insurance's life insurance;;KIOCL's iron ore mining;;Havells India's electrical equipment;;Suzlon Energy's wind power solutions;;Skipper's engineering products;;KEI Industries' cable manufacturing;;Vaibhav Global's jewelry retail;;Manappuram Finance's gold loan services;;PVR's cinema chain;;HDFC Bank's banking services;;ICICI Bank's financial services;;Tata Motors' automotive manufacturing;;Reliance Industries' conglomerate business;;Infosys' IT services;;Wipro's technology solutions;;HCL Technologies' IT consulting;;Larsen & Toubro's engineering and construction;;Mahindra & Mahindra's automotive and farm equipment;;Adani Ports and Special Economic Zone's port operations;;Sun Pharmaceutical Industries' pharmaceuticals;;Asian Paints' decorative coatings;;Bharti Aeronautical's telecom services;;ITC's diversified business;;State Bank of India's banking and financial services;;Axis Bank's banking services;;Bajaj Finance's financial services;;Hindustan Unilever's consumer goods;;Kotak Mahindra Bank's banking and financial services;;Maruti Suzuki India's automotive manufacturing;;Nestle India's food and beverage products;;Power Grid Corporation of India's electric power transmission;;Titan Company's watches and jewelry;;UltraTech Cement's cement and building materials;;Zee Entertainment Enterprises' media and entertainment;;Avenue Supermarts' retail business;;Bajaj Auto's automotive manufacturing;;Coal India's coal mining;;Dr. Reddy's Laboratories' pharmaceuticals;;Eicher Motors' automotive manufacturing;;Grasim Industries' diversified business;;HDFC's financial services;;Hero MotoCorp's motorcycle manufacturing;;Hindalco Industries' metals manufacturing;;IndusInd Bank's banking services;;JSW Steel's steel manufacturing;;NTPC's power generation;;Oil and Natural Gas Corporation's oil and gas exploration;;Shree Cement's cement manufacturing;;Tech Mahindra's IT services;;UPL's agrochemicals;;Vedanta's diversified natural resources;;YES Bank's banking services;;Ambuja Cements' cement manufacturing;;Balkrishna Industries' tire manufacturing;;Bandhan Bank's banking services;;Bharat Electronics' defense electronics;;Bharat Petroleum's petroleum refining;;Bharti Infratel's telecom tower infrastructure;;Biocon's biopharmaceuticals;;Britannia Industries' food products;;Cipla's pharmaceuticals;;Divi's Laboratories' pharmaceuticals;;DLF's real estate development;;GAIL India's natural gas processing and distribution;;Havells India's electrical equipment;;Hindustan Petroleum's petroleum refining;;ICICI Lombard General Insurance's insurance services;;Indian Oil Corporation's petroleum refining;;Indraprastha Gas' natural gas distribution;;Jindal Steel & Power's steel and power;;Lupin's pharmaceuticals;;Motherson Sumi Systems' automotive components;;Pidilite Industries' adhesives and sealants;;Piramal Enterprises' diversified business;;RBL Bank's banking services;;Siemens India's technology and engineering;;Tata Chemicals' chemicals and agri-sciences;;Tata Steel's steel manufacturing;;Torrent Pharmaceuticals' pharmaceuticals;;United Spirits' alcoholic beverages;;Aditya Birla Fashion and Retail's fashion retail;;Apollo Hospitals Enterprise's healthcare services;;Apollo Tyres' tire manufacturing;;Ashok Leyland's commercial vehicle manufacturing;;Bajaj Finserv's financial services;;Berger Paints India's paint manufacturing;;Bosch India's automotive technology;;Container Corporation of India's logistics services;;Dabur India's consumer goods;;Exide Industries' battery manufacturing;;Godrej Consumer Products' consumer goods;;Godrej Properties' real estate development;;Hindustan Zinc's zinc and lead production;;ICICI Prudential Life Insurance's life insurance;;IDFC First Bank's banking services;;Indiabulls Housing Finance's housing finance;;Indian Hotels Company's hospitality services;;Jubilant FoodWorks' food service;;JSW Energy's power generation;;L&T Finance Holdings' financial services;;L&T Technology Services' engineering and research services;;MRF's tire manufacturing;;NMDC's iron ore mining;;Page Industries' apparel manufacturing;;Prestige Estates Projects' real estate development;;Rajesh Exports' gold and diamond jewelry;;REC's power finance;;Reliance Infrastructure's infrastructure development;;SBI Cards and Payment Services' credit card services;;SBI Life Insurance's life insurance;;Shriram Transport Finance's commercial vehicle finance;;SRF's chemicals and polymers;;Tata Consumer Products' food and beverage products;;Tata Power's power generation and distribution;;Tata Communications' telecommunications services;;Thermax's energy and environment solutions;;Torrent Power's power generation and distribution;;TVS Motor Company's motorcycle manufacturing;;V-Guard Industries' electrical appliances;;Vodafone Idea's telecommunications services;;Voltas' air conditioning and engineering services;;Whirlpool of India's home appliances;;Wockhardt's pharmaceuticals;;Zydus Wellness' consumer healthcare products;;Adani Transmission's power transmission;;Adani Total Gas' natural gas distribution;;Alkem Laboratories' pharmaceuticals;;Astral Poly Technik's piping and adhesives;;Atul's chemicals and dyes;;Aurobindo Pharma's pharmaceuticals;;Bajaj Holdings & Investment's investment services;;Balkrishna Paper Mills' paper manufacturing;;Bank of Baroda's banking services;;Bharat Forge's forging and machining;;Blue Dart Express' courier and logistics services;;Canara Bank's banking services;;Cholamandalam Investment and Finance's financial services;;City Union Bank's banking services;;Colgate-Palmolive India's consumer goods;;Coromandel International's fertilizers and agrochemicals;;Cummins India's engines and power generation;;Dalmia Bharat's cement manufacturing;;DCB Bank's banking services;;Deepak Nitrite's chemicals and petrochemicals;;Dixon Technologies' electronic manufacturing services;;Edelweiss Financial Services' financial services;;Emami's consumer goods;;Equitas Small Finance Bank's banking services;;Escorts' agricultural machineryand construction equipment;;Fortis Healthcare's healthcare services;;Glenmark Pharmaceuticals' pharmaceuticals;;GMM Pfaudler's process equipment;;Godrej Agrovet's agribusiness;;Godrej Industries' diversified business;;Granules India's pharmaceuticals;;Gujarat Fluorochemicals' chemicals and refrigerants;;Gujarat State Petronet's natural gas transmission;;Happiest Minds Technologies' digital services;;Hatsun Agro Product's dairy products;;HDFC Asset Management Company's asset management;;HDFC Life Insurance's life insurance;;Hero FinCorp's financial services;;Himatsingka Seide's textile manufacturing;;Hindustan Aeronautics' aerospace and defense;;Hindustan Copper's copper mining;;Hindustan Petroleum Corporation's petroleum refining;;Home First Finance Company India's housing finance;;ICICI Securities' brokerage and financial services;;IDBI Bank's banking services;;IFCI's financial services;;IndiaMART InterMESH's online marketplace;;Indian Bank's banking services;;Indian Energy Exchange's energy trading;;Indian Railway Catering and Tourism Corporation's railway services;;Indigo Paints' paint manufacturing;;Indo Count Industries' home textiles;;Indus Towers' telecom tower infrastructure;;Info Edge India's online classifieds;;Inox Leisure's cinema exhibition;;Intellect Design Arena's software solutions;;IRB Infrastructure Developers' infrastructure development;;ITC's diversified business;;J.B. Chemicals & Pharmaceuticals' pharmaceuticals;;Jamna Auto Industries' automotive components;;JK Cement's cement manufacturing;;JK Lakshmi Cement's cement manufacturing;;Jubilant Ingrevia's life science and chemicals;;Just Dial's local search services;;Kajaria Ceramics' ceramic and vitrified tiles;;Karnataka Bank's banking services;;KEC International's power transmission and distribution;;Kotak Mahindra Asset Management Company's asset management;;L&T Infotech's IT services;;LIC Housing Finance's housing finance;;Lux Industries' innerwear and casual wear;;Mahanagar Gas' city gas distribution;;Manappuram Finance's gold loan and financial services;;Max Financial Services' life insurance and financial services;;Minda Industries' automotive components;;Motilal Oswal Financial Services' financial services;;Mphasis' IT services;;Narayana Hrudayalaya's healthcare services;;National Aluminium Company's aluminum production;;Navin Fluorine International's specialty fluorochemicals;;NCC's construction and infrastructure;;Neogen Chemicals' specialty chemicals;;Nestle India's food and beverage products;;Network18 Media & Investments' media and entertainment;;NHPC's hydroelectric power generation;;NIIT Technologies' IT services and solutions;;Nippon Life India Asset Management's asset management;;Oberoi Realty's real estate development;;Oil India's oil and gas exploration and production;;Orient Electric's electrical products;;PC Jeweller's jewelry retail;;Persistent Systems' software and IT services;;Petronet LNG's liquefied natural gas import and regasification;;Pfizer India's pharmaceuticals;;PI Industries' agrochemicals and fine chemicals;;Piramal Enterprises' diversified business;;Polycab India's wires and cables;;Poonawalla Fincorp's financial services;;Power Finance Corporation's power sector financing;;Prestige Estates Projects' real estate development;;Procter & Gamble Hygiene and Health Care's consumer goods;;Quess Corp's business services;;Radico Khaitan's alcoholic beverages;;Rain Industries' carbon products and chemicals;;Rajesh Exports' gold and diamond jewelry;;Rallis India's agrochemicals and seeds;;Ratnamani Metals & Tubes' stainless steel and carbon steel tubes;;Raymond's textile and apparel products;;Relaxo Footwears' footwear products;;Reliance Power's power generation;;Rossari Biotech's specialty chemicals;;Route Mobile's cloud communication services;;Safari Industries India's luggage and travel accessories;;Sanofi India's pharmaceuticals;;Saregama India's music and entertainment;;Schaeffler India's automotive and industrial bearings;;Sharda Cropchem's agrochemicals and chemicals;;Shilpa Medicare's pharmaceutical manufacturing;;Shree Renuka Sugars' sugar manufacturing;;Siemens India's technology and engineering;;SJVN's hydroelectric power generation;;SKF India's bearings and related products;;Snowman Logistics' cold chain logistics;;Solar Industries India's industrial explosives;;Somany Ceramics' ceramic tiles and sanitaryware;;Spandana Sphoorty Financial's microfinance services;;Star Cement's cement manufacturing;;Steel Authority of India's steel production;;Sterling and Wilson Solar's solar EPC services;;Strides Pharma Science's pharmaceuticals;;Sun Pharma Advanced Research Company's pharmaceutical research;;Sundaram Finance's financial services;;Suprajit Engineering's automotive cables;;Supreme Industries' plastic products;;Suzlon Energy's wind power solutions;;Symphony's air coolers;;Syngene International'scontract research and manufacturing;;Tata Chemicals' chemicals and agri-sciences;;Tata Communications' telecommunications services;;Tata Consumer Products' food and beverage products;;Tata Elxsi's design and technology services;;Tata Investment Corporation's investment services;;Tata Metaliks' pig iron and ductile iron pipes;;Tata Motors Finance's vehicle financing;;Tata Power's power generation and distribution;;Tata Steel Long Products' steel manufacturing;;Tata Steel's steel manufacturing;;TCNS Clothing's women's apparel;;TeamLease Services' staffing and HR services;;Tech Mahindra's IT services;;The Federal Bank's banking services;;The Great Eastern Shipping Company's shipping and offshore services;;The Indian Hotels Company's hospitality services;;The Jammu & Kashmir Bank's banking services;;The Karnataka Bank's banking services;;The New India Assurance Company's insurance services;;The Ramco Cements' cement manufacturing;;The South Indian Bank's banking services;;Thermax's energy and environment solutions;;Thomas Cook India's travel and tourism services;;Thyrocare Technologies' diagnostic and preventive healthcare services;;Tide Water Oil's lubricants and greases;;Timken India's bearings and mechanical power transmission products;;Titan Company's watches and jewelry;;Torrent Pharmaceuticals' pharmaceuticals;;Torrent Power's power generation and distribution;;Trident's textile and paper products;;Triveni Turbine's steam turbines;;Tube Investments of India's engineering and metal products;;TV18 Broadcast's media and entertainment services;;TVS Electronics' electronic products and services;;TVS Motor Company's motorcycle and scooter manufacturing;;UCO Bank's banking services;;Ujjivan Small Finance Bank's banking and financial services;;UltraTech Cement's cement and building materials;;Union Bank of India's banking services;;United Breweries' beer manufacturing;;United Spirits' alcoholic beverages;;UPL's agrochemicals;;Vakrangee's technology and retail services;;Vardhman Textiles' textile manufacturing;;Varroc Engineering's automotive components;;Vedanta's diversified natural resources;;Venky's India's poultry products;;V-Guard Industries' electrical appliances;;Vijaya Bank's banking services;;Vinati Organics' specialty chemicals;;Vindhya Telelinks' telecom cables;;VIP Industries' luggage and travel accessories;;Visaka Industries' building materials;;Voltas' air conditioning and engineering services;;VST Industries' tobacco products;;Wabco India's automotive components;;Welspun Corp's pipes and plates;;Welspun India's home textiles;;Westlife Development's fast food restaurant chain;;Whirlpool of India's home appliances;;Wipro's technology solutions;;Wockhardt's pharmaceuticals;;Wonderla Holidays' amusement parks;;Yes Bank's banking services;;Zee Entertainment Enterprises' media and entertainment services;;Zensar Technologies' IT services;;Zydus Wellness' consumer healthcare products;;ZYDUSCADILA's pharmaceuticals;;3M India's diversified technology;;ABB India's power and automation technologies;;ACC's cement manufacturing;;Adani Enterprises' diversified business;;Adani Gas' natural gas distribution;;Adani Green Energy's renewable energy;;Adani Ports and Special Economic Zone's port operations;;Adani Power's power generation;;Adani Transmission's power transmission;;Aditya Birla Capital's financial services;;Aditya Birla Fashion and Retail's fashion retail;;Aditya Birla Money's broking and advisory services;;Advanced Enzyme Technologies' enzymes and probiotics;;Aegis Logistics' oil and gas logistics;;Affle India's mobile marketing and advertising;;AIA Engineering's high chrome mill internals;;Ajanta Pharma's pharmaceuticals;;Akzo Nobel India's paints and coatings;;Alembic Pharmaceuticals' pharmaceuticals;;Alkem Laboratories' pharmaceuticals;;Allcargo Logistics' logistics services;;Amara Raja Batteries' automotive and industrial batteries;;Amber Enterprises India's air conditioning and commercial refrigeration;;Amrutanjan Health Care's healthcare products;;Anant Raj's real estate development;;Angel Broking's stock broking and wealth management;;Anup Engineering's engineering products;;APL Apollo Tubes' steel pipes and tubes;;Apollo Micro Systems' electronic systems;;Apollo Pipes' piping solutions;;Apollo Tyres' tire manufacturing;;Aptech's education and training services;;Arvind's textile and apparel manufacturing;;Asahi India Glass' automotive and architectural glass;;Ashiana Housing's real estate development;;Ashok Leyland's commercial vehicle manufacturing;;Asian Granito India's tiles and marbles;;Aster DM Healthcare's healthcare services;;Astral Poly Technik's piping and adhesives;;Atul Auto's three-wheeler manufacturing;;Atul's chemicals and dyes;;AU Small Finance Bank's banking services;;Aurobindo Pharma's pharmaceuticals;;Avenue Supermarts' retail business;;Aviation industry growth in India;;Axis Bank's banking services;;Bajaj Auto's automotive manufacturing;;Bajaj Consumer Care's consumer goods;;Bajaj Electricals' consumer appliances;;Bajaj Finance's financial services;;Bajaj Finserv's financial services;;Bajaj Holdings & Investment's investment services;;Balkrishna Industries' tire manufacturing;;Balrampur Chini Mills' sugar manufacturing;;Bandhan Bank's banking services;;Bank of Baroda's banking services;;Bank of India's banking services;;BASF India's chemicals and materials;;Bata India's footwear manufacturing;;Bayer CropScience's agrochemicals;;BEML's mining and construction equipment;;Berger Paints India's paint manufacturing;;Bharat Bijlee's electrical engineering;;Bharat Dynamics' defense equipment;;Bharat Electronics' defense electronics;;Bharat Financial Inclusion's microfinance services;;Bharat Forge's forging and machining;;Bharat Heavy Electricals' heavy electrical equipment;;Bharat Petroleum's petroleum refining;;Bharti Airtel's telecommunications services;;Bharti Infratel's telecom tower infrastructure;;BHEL's engineering and manufacturing;;Biocon's biopharmaceuticals;;Birla Corporation's cement and jute manufacturing;;Birlasoft's IT services;;Blue Dart Express' courier and logistics services;;Blue Star's air conditioning and refrigeration;;Bodal Chemicals' dyestuffs and chemicals;;Bombay Burmah Trading Corporation's tea and coffee plantations;;Bombay Dyeing & Manufacturing Company's textiles and real estate;;Bosch India's automotive technology;;Brigade Enterprises' real estate development;;Britannia Industries' food products;;Butterfly Gandhimathi Appliances' kitchen appliances;;Cadila Healthcare's pharmaceuticals;;Can Fin Homes' housing finance;;Canara Bank's banking services;;Caplin Point Laboratories' pharmaceuticals;;Carborundum Universal's abrasives and ceramics;;Castrol India's lubricants;;CEAT's tire manufacturing;;Central Bank of India's banking services;;Century Enka's synthetic yarns;;Century Plyboards' plywood and laminates;;Century Textiles & Industries' textiles and cement;;Cera Sanitaryware's bathroom fittings;;Chambal Fertilisers and Chemicals' fertilizers and chemicals;;Chennai Petroleum Corporation's petroleum refining;;Cholamandalam Investment and Finance's financial services;;Cipla's pharmaceuticals;;City Union Bank's banking services;;Clariant Chemicals India's specialty chemicals;;Coal India's coal mining;;Colgate-Palmolive India's consumer goods;;Container Corporation of India's logistics services;;Coromandel International's fertilizers and agrochemicals;;Cox & Kings' travel and tourism services;;Crisil's ratings and research;;Crompton Greaves Consumer Electricals' electrical products;;Cummins India's engines and power generation;;Cyient's engineering and technology solutions;;Dabur India's consumer goods;;DCB Bank's banking services;;Deepak Fertilisers and Petrochemicals' fertilizers and chemicals;;Deepak Nitrite's chemicals and petrochemicals;;Dewan Housing Finance Corporation's housing finance;;Dhanlaxmi Bank's banking services;;Dilip Buildcon's infrastructure construction;;Dish TV's direct-to-home services;;Divi's Laboratories' pharmaceuticals;;DLF's real estate development;;Dr. Lal PathLabs' diagnostic and healthcare services;;Dr. Reddy's Laboratories' pharmaceuticals;;Eicher Motors' automotive manufacturing;;EID Parry's sugar and nutraceuticals;;Elgi Equipments' air compressors;;Emami's consumer goods;;Endurance Technologies' automotive components;;Engineers India's engineering and consultancy services;;Equitas Holdings' financial services;;Equitas Small Finance Bank's banking services;;Escorts' agricultural machinery and construction equipment;;Essel Propack's packaging solutions;;Eveready Industries India's batteries and flashlights;;Exide Industries' battery manufacturing;;Federal Bank's banking services;;Finolex Cables' electrical and communication cables;;Finolex Industries' PVC pipes and fittings;;Firstsource Solutions' business process outsourcing;;Force Motors' automotive manufacturing;;Fortis Healthcare's healthcare services;;Future Consumer's consumer goods;;Future Retail's retail business;;GAIL India's natural gas processing and distribution;;Garden Reach Shipbuilders & Engineers' shipbuilding;;Gateway Distriparks' logistics services;;Gati's express distribution and supply chain solutions;;GE Power India's power generation equipment;;General Insurance Corporation of India's reinsurance services;;Gillette India's personal care products;;Glenmark Pharmaceuticals' pharmaceuticals;;GlaxoSmithKline Pharmaceuticals' pharmaceuticals;;Godfrey Phillips India's tobacco products;;Godrej Agrovet's agribusiness;;Godrej Consumer Products' consumer goods;;Godrej Industries' diversified business;;Godrej Properties' real estate development;;Gokaldas Exports' apparel manufacturing;;Gokul Refoils and Solvent's edible oils and agro commodities;;Granules India's pharmaceuticals;;Graphite India's graphite electrodes;;Grasim Industries' diversified business'''
macro = '''Analysis of current macroeconomic trends in India;;Impact of global economic changes on Indian markets;;Forecast of commodity market movements in India;;Evaluation of the Indian rupee's performance against major currencies;;Influence of recent RBI policies on inflation rates;;SEBI's impact on equity market regulations;;Analysis of India's trade deficit and its implications;;Effect of budget announcements on Indian financial markets;;Forecasting India's GDP growth in the current economic climate;;Evaluation of the real estate market post-budget reforms;;Implications of global oil price fluctuations on Indian economy;;GST's impact on small and medium enterprises;;Trends in foreign direct investment in India;;Analysis of India's fiscal policy and public debt;;SEBI's role in enhancing stock market transparency;;RBI's measures to stabilize the Indian banking sector;;Impact of international trade agreements on India's economy;;Assessment of India's inflation and monetary policy;;Evaluation of the Indian stock market's performance;;Analysis of India's economic recovery post-pandemic;;Implications of changing global interest rates on Indian finance;;SEBI's initiatives for investor protection;;RBI's strategies for digital banking growth in India;;Impact of commodity price volatility on Indian markets;;Analysis of India's balance of payments;;GST compliance and its impact on Indian businesses;;RBI's foreign exchange reserves and currency stability;;SEBI's approach to corporate governance in Indian companies;;Impact of taxation reforms on Indian economy;;Analysis of India's consumer spending trends;;RBI's role in financial inclusion and economic growth;;SEBI's regulatory reforms in mutual fund investments;;Assessment of India's industrial production and growth;;Impact of budgetary allocations on infrastructure development;;GST rates revision and its effect on various sectors;;RBI's credit policy and its impact on loan rates;;Analysis of India's export-import dynamics;;SEBI's guidelines on algorithmic trading;;RBI's measures to tackle non-performing assets in banks;;GST's role in streamlining indirect taxation;;SEBI's policies on market manipulation and insider trading;;Analysis of India's economic resilience in global turmoil;;Implications of recent RBI interest rate decisions;;SEBI's framework for startup funding and listings;;Assessment of India's energy sector policies;;Impact of digital payments on Indian economy;;GST's effect on the manufacturing sector;;RBI's guidelines for fintech and digital lending;;SEBI's strategies for enhancing capital market depth;;Assessment of India's agricultural sector performance;;Impact of international sanctions on India's trade;;GST impact on the service industry;;RBI's policies on managing inflation and growth;;SEBI's approach to ESG (Environmental, Social, and Governance) investing;;Analysis of India's financial market post-global recession;;RBI's initiatives for sustainable banking;;SEBI's measures against financial fraud;;Impact of climate change policies on India's economy;;GST implementation challenges and solutions;;RBI's strategies for managing public sector banks;;SEBI's role in regulating new financial products;;Assessment of India's retail sector in the current economy;;Impact of global technology trends on Indian markets;;GST's influence on consumer goods pricing;;RBI's approach to international trade finance;;SEBI's policies on enhancing market liquidity;;Analysis of India's urban development and housing market;;Impact of demographic changes on India's economic growth;;GST's role in economic formalization;;RBI's monetary policy adjustments and economic outcomes;;SEBI's initiatives in risk management for financial markets'''
keywords = '''Inflation trends;;RBI policy;;SEBI guidelines;;GDP growth;;Rupee valuation;;Commodity prices;;Budget impacts;;Market analysis;;Oil prices;;Digital currency;;Taxation reforms;;Fiscal deficit;;Equity regulations;;FDI inflows;;Public debt;;Investor protection;;Banking stability;;Trade agreements;;Inflation control;;Stock performance;;Economic recovery;;Interest rates;;Corporate governance;;Consumer spending;;Financial inclusion;;Mutual fund investment;;Industrial growth;;Infrastructure spending;;GST rates;;Loan rates;;Export-import;;Algorithmic trading;;Non-performing assets;;Indirect taxation;;Market manipulation;;Economic resilience;;Interest rate decisions;;Startup funding;;Energy policies;;Digital payments;;Manufacturing sector;;Fintech regulation;;Capital market;;Agricultural sector;;International sanctions;;Service industry;;Inflation management;;ESG investing;;Global recession;;Sustainable banking;;Financial fraud;;Climate policies;;GST challenges;;Public sector banks;;Financial products;;Retail sector;;Technology trends;;Consumer pricing;;Trade finance;;Market liquidity;;Urban development;;Demographic changes;;Economic formalization;;Monetary adjustments;;Risk management;;Repo rate;;Crude futures;;Sensex update;;Nifty insights;;Forex trends;;Tax slabs;;GST impact;;Rural economy;;Urban inflation;;Fiscal policy;;Monetary policy;;Trading tips;;Bull market;;Bear market;;Blue chips;;Bank rates;;Loan norms;;Credit score;;PE ratio;;Market cap;;Mutual funds;;Bond yields;;Dividend stocks;;Angel investing;;Startup IPOs;;Trade deficit;;Exports data;;Imports data;;SEBI rules;;RBI update;;Venture capital;;Real estate;;Housing loans;;Auto loans;;Gold rates;;Silver rates;;Crypto prices;;Blockchain tech;;Digital wallets;;E-payment systems;;Wealth management;;Retirement planning;;Insurance plans;;Health cover;;Education loans;;Mortgage rates;;Savings accounts;;Current account;;Fixed deposits;;Liquid funds;;Fintech startups;;Economic growth;;Recession fears;;Inflation index;;Deflation risk;;Consumer markets;;Retail investors;;Market volatility;;Stock picks;;Trading platforms;;Broker reviews;;Commodity trading;;Agriculture stocks;;Energy sector;;Tech stocks;;Pharma growth;;Biotech firms;;Infrastructure bonds;;Small-cap stocks;;Mid-cap gems;;Large-cap stability;;Pension funds;;Debt markets;;Currency exchange;;Forex reserves;;Remittances;;FDI policy;;Trade tariffs;;Budget leaks;;Fiscal stimulus;;Subsidy cuts;;Tax evasion;;Black money;;Wealth tax;;Capital gains;;GST filing;;GST evasion;;Input credit;;Market rally;;Short selling;;Options trading;;Futures contracts;;Candlestick charts;;Technical analysis;;Fundamental analysis;;Market sentiment;;Investor mood;;IPO buzz;;M&A news;;Corporate earnings;;Quarterly results;;Annual reports;;Balance sheets;;Profit margins;;Revenue growth;;Cost cutting;;Debt restructuring;;Credit lines;;Leverage ratios;;Risk assessment;;Asset allocation;;Portfolio mix;;Diversification;;Investment strategy;;Market timing;;Trading signals;;Stop loss;;Profit booking;;Dividend reinvestment;;Share buyback;;Corporate governance;;CEO appointments;;Boardroom battles;;Shareholder rights;;Proxy fights;;Regulatory compliance;;Audit reports;;Insider trading;;Market manipulation;;Financial scandals;;Economic forecasts;;Global markets;;Emerging markets;;Developed markets;;Market indices;;Exchange rates;;Commodity indexes;;Oil benchmarks;;Gold standards;;Metals market;;Agricultural commodities;;Energy futures;;Technology trends;;Start-up trends;;VC deals;;PE investments;;Angel networks;;Crowdfunding;;Fintech innovation;;Blockchain applications;;Cryptocurrency regulation;;Digital currencies;;Mobile banking;;Online trading;;Robo-advisors;;Wealthtech;;Insurtech;;Regtech;;Open banking;;API banking;;Digital transformation;;Cloud computing;;Big data;;AI in finance;;Machine learning;;Algorithmic finance;;Quantitative trading;;High-frequency trading;;Dark pools;;Flash crashes;;Market bubbles;;Financial crises;;Economic downturns;;Recovery signs;;Growth sectors;;Job market;;Employment trends;;Wage growth;;Income disparity;;Poverty rates;;Social impact;;Sustainable finance;;Green bonds;;ESG funds;;Climate finance;;Renewable energy;;Solar investments;;Wind power;;Electric vehicles;;Battery tech;;Smart cities;;Urban planning;;Rural development;;Agricultural tech;;Food security;;Water resources;;Environmental policy;;Sustainable practices;;Corporate responsibility;;Community investment;;Public-private partnerships;;Government bonds;;Municipal bonds;;Treasury yields;;Central banks;;Interest rate futures;;Inflation bonds;;Credit default swaps;;Derivatives market;;Leveraged buyouts;;Management buyouts;;Private equity;;Venture capital;;Angel investors;;Startup accelerators;;Incubators;;Innovation hubs;;Tech parks;;Silicon Valley;;Wall Street;;Dalal Street;;Nasdaq;;NYSE;;LSE;;Tokyo Exchange;;Shanghai Exchange;;Hong Kong Exchange;;Singapore Exchange;;Mumbai Exchange;;BSE;;NSE;;Commodity exchanges;;Forex platforms;;Currency pairs;;Dollar index;;Euro zone;;Brexit impact;;US economy;;China growth;;Japan recession;;India GDP;;Brazil markets;;Russia sanctions;;Middle East tensions;;African opportunities;;ASEAN dynamics;;NAFTA changes;;EU regulations;;WTO negotiations;;IMF loans;;World Bank projects;;OECD reports;;UN initiatives;;Climate summits;;G20 meetings;;Davos forum;;Economic summits;;Trade conferences;;Financial forums;;Investment seminars;;Startup conferences;;Tech expos;;Fintech events;;Blockchain summits;;Crypto conferences;;AI symposiums;;Data analytics;;Cybersecurity;;Privacy laws;;Data protection;;Regulatory frameworks;;Compliance software;;Audit tools;;Fraud detection;;AML measures;;KYC processes;;Identity verification;;Biometric security;;Digital ID;;Payment gateways;;Mobile payments;;Contactless transactions;;E-wallets;;Prepaid cards;;Credit facilities;;Lending platforms;;Peer-to-peer lending;;Microfinance;;Crowdlending;;Alternative finance;;Social lending;;Impact investing;;Microcredit;;Financial inclusion;;Unbanked populations;;Underbanked communities;;Financial literacy;;Education initiatives;;Skill development;;Youth empowerment;;Women in finance;;Diversity in business;;Inclusive growth;;Equal opportunity;;Gender equality;;Minority representation;;Social justice;;Ethical practices;;Human rights;;Labor standards;;Worker rights;;Fair trade;;Sustainable supply chains;;Corporate philanthropy;;Charitable giving;;Nonprofit funding;;Social entrepreneurship;;Community projects;;Volunteering;;Civic engagement;;Public service;;Government policies;;Regulatory bodies;;Legislative changes;;Political debates;;Election impacts;;Policy decisions;;Government spending;;Public finance;;Budget deficits;;National debt;;Fiscal balance;;Economic planning;;Development goals;;Infrastructure projects;;Public works;;Transport networks;;Communication infrastructure;;Energy grids;;Water systems;;Waste management;;Recycling initiatives;;Renewable resources;;Sustainable energy;;Clean technology;;Green tech;;Eco-friendly solutions;;Sustainability goals;;Carbon footprint;;Greenhouse gases;;Climate action;;Environmental stewardship;;Conservation efforts;;Wildlife protection;;Natural habitats;;Ecosystem preservation;;Biodiversity conservation;;Marine resources;;Ocean conservation;;Freshwater conservation;;Land management;;Forest preservation;;Agricultural sustainability;;Food systems;;Nutrition security;;Hunger relief;;Poverty alleviation;;Economic empowerment;;Community development;;Rural upliftment;;Urban renewal;;Slum upgrading;;Housing affordability;;Homeownership;;Rental markets;;Real estate development;;Property investment;;Land rights;;Urban planning;;City development;;Smart growth;;Sustainable cities;;Urban resilience;;Disaster recovery;;Emergency response;;Crisis management;;Humanitarian aid;;Relief efforts;;Refugee support;;Migration challenges;;Global health;;Public health;;Healthcare access;;Medical services;;Disease prevention;;Vaccination campaigns;;Maternal health;;Child welfare;;Education access;;Schooling initiatives;;Literacy programs;;Vocational training;;Employment opportunities;;Job creation;;Workforce development;;Skill training;;Entrepreneurship support;;Business incubation;;Startup financing;;Small business development;;Microenterprise support;;Market access;;Trade facilitation;;Export promotion;;Import substitution;;Industrial policy;;Manufacturing growth;;Technology transfer;;Innovation ecosystems;;Research and development;;Science and technology;;Engineering advances;;Mathematics and statistics;;Academic research;;University collaborations;;Educational partnerships;;Scholarship programs;;Student exchanges;;Cultural exchanges;;Arts and culture;;Creative industries;;Media and entertainment;;Film and television;;Music and performing arts;;Literature and publishing;;Visual arts;;Design and fashion;;Architecture and urbanism;;Historical preservation;;Cultural heritage;;Tourism development;;Travel and leisure;;Hospitality industry;;Recreational activities;;Sports and fitness;;Wellness and lifestyle;;Health and beauty;;Personal care;;Consumer goods;;Retail markets;;Shopping trends;;E-commerce platforms;;Online shopping;;Digital marketplaces;;Mobile commerce;;Social media;;Digital marketing;;Online advertising;;Content creation;;Influencer marketing;;Brand strategies;;Marketing analytics;;Consumer insights;;Customer engagement;;Customer service;;Client relations;;Sales strategies;;Business development;;Market expansion;;Global outreach;;International business;;Cross-border trade;;Multinational corporations;;Global brands;;Corporate alliances;;Strategic partnerships;;Joint ventures;;Mergers and acquisitions;;Corporate restructuring;;Business transformations;;Organizational change;;Leadership development;;Management practices;;Human resources;;Talent acquisition;;Employee retention;;Workplace culture;;Team building;;Organizational learning;;Training and development;;Professional growth;;Career advancement;;Networking opportunities;;Mentorship programs;;Coaching and guidance;;Personal development;;Life skills;;Self-improvement;;Motivation and inspiration;;Success stories;;Achievement and recognition;;Awards and honors;;Celebrity endorsements;;Public figures;;Influential leaders;;Thought leaders;;Industry experts;;Keynote speakers;;Panel discussions;;Roundtable debates;;Workshops and trainings;;Webinars and online courses;;E-learning platforms;;Distance education;;Remote learning;;Virtual classrooms;;Online resources;;Digital tools;;Software solutions;;Technology platforms;;Mobile apps;;Gaming and entertainment;;Virtual reality;;Augmented reality;;Artificial intelligence;;Machine learning;;Robotics and automation;;Internet of Things;;Wearable technology;;Smart devices;;Connected homes;;Energy efficiency;;Sustainable transportation;;Electric vehicles;;Autonomous vehicles;;Public transit;;Cycling and walking;;Active transportation;;Shared mobility;;Ridesharing;;Carsharing;;Bikesharing;;Scootersharing;;Public bicycles;;Urban mobility;;Traffic management;;Parking solutions;;Road safety;;Transport planning;;Mobility as a Service;;Transport innovation;;Logistics and supply chain;;Freight transport;;Shipping and maritime;;Air transport;;Rail transport;;Bus transport;;Taxi services;;Delivery services;;Courier services;;Postal services;;Package handling;;Warehousing and storage;;Inventory management;;Order fulfillment;;Supply chain optimization;;Global logistics'''
timely = '''Bitcoin price today;;Stock market today;;Fed rate decision today;;Earnings reports this week;;IPOs this week;;Crude oil prices today;;Tech stock movements today;;Gold rate today;;Currency exchange rates today;;Latest mutual fund NAVs;;Today's top gainers in stock market;;Today's top losers in stock market;;New government policy announcements;;Breaking economic news;;Latest inflation rate announcement;;Unemployment data release today;;Central bank's latest statement;;Today's forex market analysis;;Commodity market updates today;;Financial tech startup news;;Banking sector updates today;;Insurance market today;;Pension fund returns today;;Corporate earnings released today;;Trade agreement news today;;Venture capital funding news;;Private equity investment news;;Hedge fund performance today;;Global economic update today;;Market volatility alerts today;;Dividend declaration news;;Credit rating changes today;;Blockchain adoption news;;Digital payment trends today;;Artificial intelligence finance news;;Cloud computing in finance today;;Sustainable investing news;;Green bond issuances today;;Impact investing news today;;Crowdfunding campaign launches;;Peer-to-peer lending rates today;;Wealth management insights today;;Tax law changes today;;Retirement plan updates today;;Budgeting app updates;;Debt management news today;;Forex trading strategies today;;Commodity outlook today;;Precious metals market today;;Real estate investment news today;;ETF market today;;Index fund performance today;;Financial planning tips today;;Cryptocurrency mining news;;Initial coin offerings today;;Security token offerings news;;Cryptocurrency wallet updates;;Bitcoin futures market today;;Ethereum network news today;;Ripple technology updates today;;Litecoin trading today;;Blockchain startup launches;;Financial regulation updates today;;Bankruptcy news today;;Mergers and acquisitions today;;Antitrust case news today;;Corporate governance developments today;;Insider trading reports today;;Fraud detection tech news today;;Risk management tools today;;Insurance tech updates today;;Robo-advisor news today;;Mobile banking features today;;Neobanks news today;;Payment gateway developments today;;Credit card program launches today;;Fintech accelerator news today;;Digital currency exchange news today;;Smart contract developments today;;Decentralized finance (DeFi) news today;;IPO strategies today;;SPAC news today;;CSR initiatives today;;Economic stimulus news today;;Government budget news today;;Trade tariff updates today;;IMF loan news today;;World Bank project updates today;;Sovereign wealth fund activities today;;FDI trends today;;Export credit news today;;International trade stats today;;Emerging economies today;;Economic sanctions news today;;Global supply chain news today;;Outsourcing trends today;;Manufacturing data today;;Automobile sales news today;;Pharmaceutical industry news today;;Tech sector investments today;;Media industry earnings today;;Tourism industry updates today;;Hospitality recovery news today;;Agriculture market news today;;Food industry trends today;;Transportation news today;;Shipping industry today;;Aviation sector updates today;;Railway developments today;;Infrastructure financing news today;;Renewable energy projects today;;Oil and gas news today;;Mining industry updates today;;Chemical industry news today;;Steel production news today;;Textile industry today;;Consumer electronics market today;;Gaming industry revenue today;;Streaming service competition today;;Social media earnings today;;E-commerce expansions today;;Online advertising trends today;;Digital content creation news today;;Cybersecurity solutions today;;Data privacy regulation news today;;Cloud storage options today;;SaaS growth news today;;IoT applications today;;Virtual reality investments today;;Augmented reality projects today;;AI breakthroughs today;;Machine learning developments today;;Big data analytics news today;;Quantum computing research today;;Nanotechnology finance news today;;Biotechnology financing today;;Healthcare innovations today;;Medical device market today;;Pharmaceutical research funding today;;Wellness tech news today;;EdTech startup news today;;Online learning platforms today;;Remote work solutions today;;Gig economy impact today;;Unemployment insurance news today;;Minimum wage updates today;;Labor market trends today;;Workforce development news today;;Employee benefits innovation today;;Diversity and inclusion initiatives today;;Gender pay gap analysis today;;Workplace safety standards today;;Remote work policies today;;Freelancing news today;;Entrepreneurship ecosystem news today;;Small business growth strategies today;;E-commerce for small businesses today;;Social entrepreneurship news today;;Non-profit financial management today;;Microfinance institutions today;;Community development financial news today;;Impact measurement news today;;Corporate bond market today;;Government securities today;;Municipal bonds today;;Derivatives trading news today;;Options and futures today;;Leveraged and inverse ETFs today;;Currency hedging today;;Commodity hedging today;;Financial derivatives today;;Asset-backed securities today;;Mortgage-backed securities today;;Collateralized debt obligations today;;Credit default swaps today;;Interest rate swaps today;;Foreign exchange swaps today;;Structured finance products today;;Capital markets innovation today;;Electronic trading platforms today;;Algorithmic trading news today;;High-frequency trading (HFT) news today;;Dark pools of liquidity today;;Securities lending news today;;Clearing and settlement processes today;;Financial market infrastructure news today;;Stock exchange updates today;;Regulatory technology (RegTech) news today;;Compliance automation today;;Anti-money laundering (AML) solutions today;;Know your customer (KYC) technologies today;;Fraud prevention tools today;;Cyber insurance news today;;Financial crime investigations today;;Economic crime trends today;;Counterfeit currency detection today;;Intellectual property rights enforcement today;;Trade secret protection today;;Patent financing news today;;Stock market opening prices;;Dow Jones Industrial Average today;;NASDAQ composite index now;;S&P 500 latest update;;Bitcoin price movement today;;Ethereum trading volume 24h;;Crude oil price changes today;;Gold rate fluctuations today;;Latest IPO filings;;New merger announcements today;;Tech earnings reports released today;;Central bank interest rate decision;;Currency exchange rate today;;Fed's latest monetary policy;;Today's top-performing mutual funds;;Daily commodity market trends;;Real-time economic data releases;;Latest consumer spending reports;;Unemployment rate update today;;Today's inflation figures release;;Bank earnings announcements today;;Insurance companies quarterly results today;;Venture capital deals closed today;;Today's significant stock buybacks;;Corporate dividend announcements today;;Latest trade agreement impacts;;Emerging markets index today;;Today's highest volume trades;;Real estate market daily update;;Bond market movements today;;Automotive sales figures released today;;Renewable energy sector news today;;Retail industry earnings today;;Inflation rate news today;;Latest unemployment data;;Central bank's statement on inflation;;Currency strength today;;Commodity prices live update;;Financial tech startups funding today;;Banking sector regulation news today;;Latest insurance policy changes;;Pension fund current performance;;Corporate earnings highlights today;;Global trade news today;;Venture capital investment news today;;Private equity firm acquisitions today;;Hedge fund today's performance;;Global economic outlook today;;Market volatility index today;;Today's dividend yield updates;;Credit rating agency latest reports;;Blockchain in finance news today;;Digital payment trends today;;AI advancements in finance today;;Cloud computing finance applications today;;Sustainable investing news today;;Green bond market today;;Impact investing latest trends;;Crowdfunding platform launches today;;Peer-to-peer lending rates today;;Wealth management insights today;;Tax law changes announced today;;Retirement planning updates today;;Budgeting software new features;;Debt management strategies today;;Forex trading pair movements today;;Latest commodity trading strategies;;Precious metals market analysis today;;Real estate investment opportunities today;;ETF trading strategies today;;Index fund today's performance;;Financial planning advice today;;Cryptocurrency mining news today;;Security token offering updates today;;Cryptocurrency wallet new features;;Bitcoin futures market today;;Ethereum network updates today;;Ripple technology advancements today;;Litecoin market analysis today;;Blockchain startup launches today;;Regulatory updates in finance today;;Bankruptcy filings updates today;;Mergers and acquisitions news today;;Antitrust case developments today;;Corporate governance news today'''

In [6]:
print(f"abstract queries: {len(abstract.split(';;'))}, companies: {len(companies.split(';;'))}, macro related: {len(macro.split(';;'))}, keywords: {len(keywords.split(';;'))}, timely: {len(timely.split(';;'))}")

abstract queries: 252, companies: 697, macro related: 71, keywords: 632, timely: 284


In [7]:
abstract_queries = [x.lower() for x in abstract.split(';;')]
companies_queries = [x.lower() for x in companies.split(';;')]
macro_queries = [x.lower() for x in macro.split(';;')]
keywords_queries = [x.lower() for x in keywords.split(';;')]
timely_queries = [x.lower() for x in timely.split(';;')]

In [8]:
non_financial='''Salman Khan latest movie;;Virat Kohli batting record;;Olympics India medals;;Narendra Modi speech;;Chicken biryani recipe;;Sachin Tendulkar records;;Yoga poses for beginners;;Shahrukh Khan new film;;MS Dhoni retirement news;;Paneer tikka masala recipe;;Indian politics debate;;Rohit Sharma century highlights;;Funny cat videos;;Rajinikanth next project;;PV Sindhu match schedule;;Amitabh Bachchan awards;;Mango lassi recipe;;Cricket World Cup updates;;Kareena Kapoor fitness secrets;;Climate change effects;;Tiger Shroff dance moves;;Badminton championship winners;;Pav bhaji recipe;;Rahul Gandhi interview;;Saina Nehwal training tips;;Alia Bhatt fashion trends;;Kabaddi league results;;Gulab jamun recipe;;IPL auction highlights;;Priyanka Chopra Hollywood news;;Chess grandmaster strategies;;Butter chicken recipe;;Lok Sabha election predictions;;Virat Kohli workout routine;;Hockey World Cup highlights;;Vegetable biryani recipe;;Sunil Chhetri goals;;Bollywood box office;;Neeraj Chopra javelin throw;;Chole bhature recipe;;Deepika Padukone movies;;T20 league updates;;Samosa recipe;;Akshay Kumar stunts;;Parliament session updates;;Anushka Sharma beauty tips;;Football transfer news;;Rasgulla recipe;;Ranveer Singh performances;;Political rally speeches;;Saina Nehwal vs PV Sindhu;;Dal makhani recipe;;Arijit Singh concerts;;Olympic qualification India;;Paneer butter masala recipe;;Lionel Messi vs Cristiano Ronaldo;;Shahid Kapoor fitness regime;;Supreme Court verdicts;;Kidambi Srikanth matches;;Tandoori chicken recipe;;Ajay Devgn upcoming films;;Wrestling championship India;;Masala dosa recipe;;Indian Idol auditions;;Tennis Grand Slam winners;;Fish curry recipe;;Ranbir Kapoor new look;;Badminton training drills;;Pani puri recipe;;Saif Ali Khan cooking show;;Marathon running tips;;Chocolate cake recipe;;Katrina Kaif dance workout;;Squash tournament schedule;;Palak paneer recipe;;John Abraham action scenes;;Cricket batting techniques;;Naan recipe;;Varun Dhawan comedy scenes;;Table tennis techniques;;Matar paneer recipe;;Hrithik Roshan diet plan;;Formula 1 race updates;;Idli sambar recipe;;Kriti Sanon acting tips;;Cycling race champions;;Carrot halwa recipe;;Bollywood celebrity weddings;;Football World Cup favorites;;Ladoo recipe;;Ayushmann Khurrana songs;;Hockey goalkeeper drills;;Jalebi recipe;;Jacqueline Fernandez workout;;Chess opening moves;;Rajma recipe;;Shraddha Kapoor singing;;Kushti traditional wrestling;;Malai kofta recipe;;Cricket sledging moments;;Bollywood dance steps;;Mushroom masala recipe;;Biography of Virat Kohli;;Tennis ace techniques;;Poha recipe;;Indian stand-up comedy;;Gymnastics Olympic trials;;Sponge cake recipe;;Taapsee Pannu thriller movies;;Athletics training regimen;;Aloo paratha recipe;;Bollywood film reviews;;Badminton smash technique;;Gajar ka halwa recipe;;Yo-Yo test in cricket;;Sportsmanship stories;;Momos recipe;;Dangal movie insights;;Motivational sports quotes;;Sarson da saag recipe;;Bollywood fashion icons;;Kabbadi defensive tactics;;Besan ladoo recipe;;Reality TV show auditions;;Cricket umpiring signals;;Rasmalai recipe;;Political satire shows;;Olympic torch history;;Shahi paneer recipe;;Adventure sports in India;;Stand-up paddleboarding tips;;Kheer recipe;;Celebrity fitness secrets;;Mountain biking trails;;Ice cream recipe;;Funny Bollywood dialogues;;Rock climbing basics;;Pizza recipe;;Bollywood vs Hollywood;;Surfing in Goa;;Biryani varieties;;Cute animal memes;;Paragliding safety;;Chocolate mousse recipe;;Top Bollywood villains;;Skateboarding tricks;;Spaghetti recipe;;Hilarious cricket moments;;Street food secrets;;Kayaking techniques;;Brownie recipe;;Indian classical music maestros;;Skiing destinations India;;Banana bread recipe;;Epic Bollywood dance-offs;;Marathon nutrition plan;;Cheesecake recipe;;Famous Bollywood playback singers;;Triathlon training schedule;;Pancake recipe;;Blockbuster Bollywood dialogues;;Chess strategy for beginners;;Strawberry shortcake recipe;;Bollywood choreography tutorials;;Boxing weight classes;;Omelette recipe;;Iconic cricket matches;;Healthy smoothie recipes;;Bollywood themed parties;;Zumba workout for beginners;;Apple pie recipe;;Cricket trivia and facts;;Homemade pasta recipe;;Stand-up comedy specials;;Yoga for stress relief;;Brownie recipe;;Celebrity talk show moments;;DIY home decor ideas;;Nutella crepes recipe;;Parody Bollywood songs;;DIY fashion hacks;;Tiramisu recipe;;Classic Indian jokes;;Sustainable living tips;;Matcha tea benefits;;Upcycling projects for home;;Falafel recipe;;Laugh out loud pranks;;Eco-friendly products;;Quinoa salad recipe;;Viral dance challenges;;Natural beauty remedies;;Lemon tart recipe;;Comedy night lineup;;Herbal tea recipes;;Street style fashion;;Pumpkin soup recipe;;Sketch comedy clips;;Detox water recipes;;Vintage Bollywood posters;;Hot chocolate recipe;;Meme culture in India;;Vegan diet basics;;Sourdough bread recipe;;Popular web series;;Gluten-free recipes;;Homemade pizza dough;;Travel vloggers in India;;Keto diet meals;;Veggie burger recipe;;Bollywood trivia quizzes;;Plant-based protein sources;;Stuffed paratha recipe;;Social media influencers;;Organic gardening tips;;Mango smoothie recipe;;Youth empowerment initiatives;;Creative writing prompts;;Mixed berry compote recipe;;Public speaking tips;;Homemade ice cream flavors;;Photography contest themes;;Cucumber sandwich recipe;;E-learning resources;;Cocktail mixing techniques;;DIY beauty treatments;;Spicy chicken wings recipe;;Podcast recommendations;;Juicing for health;;Spinach quiche recipe;;Cultural festivals in India;;Herb gardening basics;;Chocolate chip cookie recipe;;Art and craft tutorials;;Fruit tart recipe;;Binge-worthy TV shows;;Culinary workshop experiences;;Avocado toast variations;;Poetry slam events;;Breakfast smoothie bowls;;Cultural heritage sites;;Veggie stir-fry recipe;;Music festival lineups;;Homemade granola bars;;Historical landmarks tours;;Tomato soup recipe;;Contemporary dance moves;;Green smoothie recipes;;Art exhibition openings;;Banana pancake recipe;;Literary cafe readings;;Refreshing salad recipes;;Street art tours;;Caramel latte recipe;;Book club selections;;Detox salad recipes;;Open mic nights;;Custom cake designs;;Motivational workout playlists;;Baking basics class;;Summer fruit salad;;Film photography tips;;Classic margarita recipe;;DIY home workout equipment;;Roasted vegetable recipes;;Sustainable fashion brands;;Pasta primavera recipe;;Mindfulness meditation guides;;Grilled cheese variations;;Eco-tourism spots;;Chickpea curry recipe;;Creative journaling ideas;;Sweet potato fries recipe;;Thrifting and vintage shopping;;Lentil soup recipe;;Indie music artists;;Mushroom risotto recipe;;Community service projects;;Vegan chocolate cake;;Upcoming art fairs;;Tofu scramble recipe;;Freelancing tips for beginners;;Quiche Lorraine recipe;;Yoga retreats in India;;Vegetarian chili recipe;;Start-up pitch events;;Fruit popsicle recipes;;Digital detox strategies;;Vegetable lasagna recipe;;Ted Talks on innovation;;Cucumber gazpacho recipe;;Cultural exchange programs;;Black bean burger recipe;;Personal development workshops;;Apple crisp recipe;;Outdoor adventure clubs;;Vegetarian sushi rolls;;Networking events for creatives;;Homemade jam recipes;;Women's leadership forums;;Zucchini noodle recipe;;Philanthropy and charity galas;;Peanut butter cookie recipe;;Youth leadership conferences;;Stuffed bell peppers recipe;;Artisanal craft markets;;Chocolate fondue recipe;;Eco-friendly living workshops;;Basil pesto pasta recipe;;Photography exhibitions;;Oatmeal raisin cookies recipe;;Cultural appreciation societies;;Lemon meringue pie recipe;;Innovative tech meetups;;Spinach and feta pie recipe;;Social entrepreneurship forums;;Carrot cake recipe;;Creative writing workshops;;Pumpkin pie recipe;;Film screening and discussion panels;;Ratatouille recipe;;Music composition classes;;Sweet and sour tofu recipe;;Art therapy sessions;;Fluffy pancake recipe;;Entrepreneurial bootcamps;;Tomato bruschetta recipe;;Professional development seminars;;Peach cobbler recipe;;Outdoor photography walks;;Vegetable tempura recipe;;Literary magazine launches;;Strawberry cheesecake recipe;;Community gardening initiatives;;Red velvet cake recipe;;Public speaking workshops;;Mediterranean salad recipe;;Artisan bread baking;;Mint mojito recipe;;Cultural dance classes;;Spicy tofu stir-fry recipe;;Youth mentorship programs;;Homemade salsa recipe;;Environmental conservation talks;;Blueberry muffins recipe;;Career counseling sessions;;Cherry pie recipe;;Social justice panels;;Vegetable spring rolls recipe;;Mental health awareness events;;Key lime pie recipe;;Networking mixers for professionals;;Raspberry sorbet recipe;;Film appreciation clubs;;Garlic bread recipe;;Civic engagement forums;;Pineapple upside-down cake recipe;;Leadership development programs;;Mac and cheese recipe;;Innovation and design thinking workshops;;Lemon bars recipe;;Outdoor survival skills training;;Chocolate lava cake recipe;;Entrepreneurship case studies;;Banana bread recipe'''

In [9]:
len(non_financial.split(';;'))

365

In [10]:
non_financial_queries = [x.lower() for x in non_financial.split(';;')]

In [11]:
all_queries = abstract_queries + companies_queries + macro_queries + keywords_queries + timely_queries + non_financial_queries + insight_queries

In [12]:
len(all_queries)

2863

In [13]:
len(set(all_queries))

2727

In [14]:
unique_queries = list(set(all_queries))

#### Save queries to file

In [25]:
# with open('queries.txt','w') as tfile:
# 	tfile.write('\n'.join(all_queries))

In [26]:
from src.QueryService import QueryService

### Getting query understanding

In [23]:
with open('queries.txt', 'r') as tfile:
    all_queries = tfile.read().split('\n')

In [27]:
unique_queries = list(set(all_queries))

In [28]:
import numpy as np

In [29]:
np.random.shuffle(unique_queries)

In [30]:
unique_queries

['bcci',
 'securities lending news today',
 'social media influencers',
 "the new india assurance company's insurance services",
 "ujjivan small finance bank's banking and financial services",
 'rock climbing basics',
 'security token offerings news',
 'tata consumer share price live',
 "irctc's rail ticketing and catering services",
 'pension fund returns today',
 'sbi life insurance stock analysis',
 'gold investment',
 'sensex daily performance',
 "kotak mahindra bank's banking and financial services",
 'regulatory frameworks',
 'loans',
 'current status of indian steel industry',
 'asian paints stock price',
 "the ramco cements' cement manufacturing",
 'disaster recovery',
 'best savings account interest rates in india',
 'dividend stocks',
 'online trading',
 'economic stimulus news today',
 'latest fdi policy india',
 'shahi paneer recipe',
 'taapsee pannu thriller movies',
 "bharat heavy electricals' heavy electrical equipment",
 'brand strategies',
 'sunpharma share price live'

In [31]:
def understand_query(query):
    random_days_ago = np.random.randint(-15,45)
    today = datetime.now()
    dt = (today - timedelta(days=random_days_ago)).strftime('%d/%m/%y')
    is_fin, recency, new_query = QueryService.understand_query_using_llm(query,query_date=dt)
    return query, dt, is_fin, recency, new_query

In [32]:
understand_query('which stock is good to invest')

query understanding llm in 0.9275338649749756 seconds


('which stock is good to invest',
 '17/02/24',
 True,
 'high',
 'Best stocks to invest in 2024')

In [20]:
# understand_query('sensex today')

In [7]:
import logging

In [8]:
logger = logging.getLogger('__name__')
level = logging.ERROR
logger.setLevel(level)

In [9]:
import time
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
import requests

In [36]:
query_understanding = {}

In [37]:
chunk_size = 200
num_chunks = len(unique_queries)//chunk_size + 1

In [38]:
for i in range(num_chunks):
    query_chunk = unique_queries[i*chunk_size:(i+1)*chunk_size]
    print(f'starting creation at {"{:%b %d, %Y %H:%M}".format(datetime.now())}')
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=15) as executor:
        futures = [executor.submit(understand_query, query) for query in query_chunk]

        for future in as_completed(futures):
            response = future.result()
            if response:  # Timeout or other errors
                query_understanding[response[0]] = {'query_date': response[1], 'finance_query': response[2], 'recency_importance': response[3], 'improved_query': response[4]}
    print(f'completed chunk {(i + 1) * chunk_size} queries at {"{:%b %d, %Y %H:%M}".format(datetime.now())} in {int(time.time() - start_time)/60} minutes')

starting creation at Feb 05, 2024 16:17
completed chunk 200 queries at Feb 05, 2024 16:17 in 0.55 minutes
starting creation at Feb 05, 2024 16:17
completed chunk 400 queries at Feb 05, 2024 16:18 in 0.5333333333333333 minutes
starting creation at Feb 05, 2024 16:18
completed chunk 600 queries at Feb 05, 2024 16:18 in 0.55 minutes
starting creation at Feb 05, 2024 16:18
completed chunk 800 queries at Feb 05, 2024 16:19 in 0.5666666666666667 minutes
starting creation at Feb 05, 2024 16:19
completed chunk 1000 queries at Feb 05, 2024 16:20 in 0.6833333333333333 minutes
starting creation at Feb 05, 2024 16:20
completed chunk 1200 queries at Feb 05, 2024 16:20 in 0.55 minutes
starting creation at Feb 05, 2024 16:20
completed chunk 1400 queries at Feb 05, 2024 16:21 in 0.55 minutes
starting creation at Feb 05, 2024 16:21
completed chunk 1600 queries at Feb 05, 2024 16:21 in 0.5333333333333333 minutes
starting creation at Feb 05, 2024 16:21
completed chunk 1800 queries at Feb 05, 2024 16:22 i

In [10]:
import json

In [11]:
# with open('query_understanding.json', 'w') as f:
#     json.dump(query_understanding,f)

In [12]:
with open('query_understanding.json', 'r') as f:
    query_understanding = json.load(f)

In [13]:
import pandas as pd

In [14]:
queries_df = pd.DataFrame.from_dict(query_understanding).T.reset_index().rename(columns = {'index': 'query'})

In [15]:
queries_df.sample(40)

,query,query_date,finance_query,recency_importance,improved_query
1895,tata steel share price live,26/01/24,True,high,Tata Steel share price live on 26 January 2024
2717,how to pan tax filings,13/02/24,True,medium,How to PAN tax filings in India 2024
2646,lending platforms,23/01/24,True,high,Lending platforms in India 2024
652,blockchain technology in indian banking,11/01/24,True,medium,Blockchain technology in Indian banking sector...
1481,sustainable finance,21/01/24,True,medium,Sustainable finance trends in 2024
1692,indian online gaming industry analysis,10/02/24,True,medium,Indian online gaming industry analysis 2024
422,reality tv show auditions,01/01/24,True,high,Reality TV show auditions in India 2024
2326,hindalco share price live,16/01/24,True,high,Hindalco share price on 16 January 2024
2145,tips for buying stocks in india,03/02/24,True,low,Best tips for stock investment in India 2024
338,reliance industries' conglomerate business,07/02/24,True,medium,Reliance Industries conglomerate business in 2024


In [16]:
queries_df.groupby('recency_importance')['query'].count()

recency_importance
high       634
low        171
medium    1718
Name: query, dtype: int64

In [17]:
queries_df.groupby('finance_query')['query'].count()

finance_query
False     198
True     2529
Name: query, dtype: int64

### Generating search results

In [18]:
from datetime import timedelta

import json

In [19]:
def get_top_hits_for_query_row(query_row):
    if query_row.finance_query:
        query_date = datetime.strptime(query_row.query_date,'%d/%m/%y') + timedelta(days=1,seconds=-1)
        formatted_date = query_date.strftime('%Y-%m-%dT%H:%M:%S')
        if query_row.improved_query:
            # If the date is a future date, then change importance to medium to get articles from ES
            if query_row.recency_importance == 'high' and query_date >= datetime.now():
                results_object = HybridSearchService.perform_hybrid_reranked_search(query_text = query_row.improved_query, recency_importance='medium',max_published_date=formatted_date)
            else:
                results_object = HybridSearchService.perform_hybrid_reranked_search(query_text = query_row.improved_query, recency_importance=query_row.recency_importance,max_published_date=formatted_date)
        else:
            results_object = HybridSearchService.perform_hybrid_reranked_search(query_text = query_row.query, recency_importance='medium', max_published_date=formatted_date)
        top_hits = [x['article_id'] for x in results_object['searchArticleIds']][:10]
        return query_row.name, query_row.query, query_row.query_date, json.dumps(top_hits)
    else:
        return query_row.name, query_row.query, query_row.query_date, json.dumps([])

In [20]:
get_top_hits_for_query_row(queries_df.iloc[5])

(5,
 "kotak mahindra bank's banking and financial services",
 '23/12/23',
 '["657fb5c98f821120f67a0925", "65780584b8b4d445277abb85", "65783d471678087a992963e4", "657aa8113a7a250d32265347", "6577cd31b8b4d445277abacd", "6565c85739bfc8784efe5a24", "656e02539994965ac4b4b47c", "6557482a4b13023f934aef36", "6571c30925cc9a343f03d505", "6555cc274b13023f934a59fe"]')

In [21]:
queries_df.head(10)

,query,query_date,finance_query,recency_importance,improved_query
0,regulatory frameworks,27/01/24,True,medium,Regulatory frameworks in India 2024
1,security token offerings news,05/02/24,True,high,Security token offerings news in 2024
2,gold investment,01/01/24,True,medium,Gold investment trends in 2024
3,pension fund returns today,14/02/24,True,high,Pension fund returns on 14 February 2024
4,social media influencers,16/02/24,True,medium,Social media influencers trends in 2024
5,kotak mahindra bank's banking and financial se...,23/12/23,True,medium,Kotak Mahindra Bank's financial and business n...
6,sbi life insurance stock analysis,25/01/24,True,high,SBI Life Insurance stock analysis 2024
7,rock climbing basics,28/12/23,True,medium,Rock climbing basics in India 2024
8,securities lending news today,25/01/24,True,high,Securities lending news January 25 2024
9,bcci,18/01/24,True,medium,BCCI financial and business news 2024


In [22]:
search_result_articles = []

In [23]:
chunk_size = 10
num_chunks = len(queries_df)//chunk_size + 1

In [24]:
num_chunks

273

In [25]:
for i in range(num_chunks):
    query_chunk = queries_df.iloc[i*chunk_size:(i+1)*chunk_size]
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=30) as executor:
        futures = [executor.submit(get_top_hits_for_query_row, row) for index, row in query_chunk.iterrows()]
        for future in as_completed(futures):
            response = future.result()
            if response:  # Timeout or other errors
                search_result_articles.append(response)
    print(f'completed chunk {(i + 1) * chunk_size} queries at {"{:%b %d, %Y %H:%M}".format(datetime.now())} in {int(time.time() - start_time)/60} minutes')

completed chunk 10 queries at Feb 05, 2024 21:08 in 0.38333333333333336 minutes
completed chunk 20 queries at Feb 05, 2024 21:08 in 0.23333333333333334 minutes


completed chunk 30 queries at Feb 05, 2024 21:08 in 0.2 minutes
completed chunk 40 queries at Feb 05, 2024 21:09 in 0.23333333333333334 minutes
completed chunk 50 queries at Feb 05, 2024 21:09 in 0.23333333333333334 minutes
completed chunk 60 queries at Feb 05, 2024 21:09 in 0.23333333333333334 minutes
completed chunk 70 queries at Feb 05, 2024 21:09 in 0.21666666666666667 minutes
completed chunk 80 queries at Feb 05, 2024 21:10 in 0.23333333333333334 minutes
completed chunk 90 queries at Feb 05, 2024 21:10 in 0.25 minutes
completed chunk 100 queries at Feb 05, 2024 21:10 in 0.21666666666666667 minutes
completed chunk 110 queries at Feb 05, 2024 21:10 in 0.16666666666666666 minutes
completed chunk 120 queries at Feb 05, 2024 21:11 in 0.18333333333333332 minutes
completed chunk 130 queries at Feb 05, 2024 21:11 in 0.25 minutes
completed chunk 140 queries at Feb 05, 2024 21:11 in 0.2 minutes
completed chunk 150 queries at Feb 05, 2024 21:11 in 0.21666666666666667 minutes
completed chunk 

completed chunk 360 queries at Feb 05, 2024 21:16 in 0.21666666666666667 minutes
completed chunk 370 queries at Feb 05, 2024 21:16 in 0.21666666666666667 minutes
completed chunk 380 queries at Feb 05, 2024 21:16 in 0.18333333333333332 minutes
completed chunk 390 queries at Feb 05, 2024 21:16 in 0.23333333333333334 minutes
completed chunk 400 queries at Feb 05, 2024 21:17 in 0.23333333333333334 minutes
completed chunk 410 queries at Feb 05, 2024 21:17 in 0.11666666666666667 minutes
completed chunk 420 queries at Feb 05, 2024 21:17 in 0.23333333333333334 minutes
completed chunk 430 queries at Feb 05, 2024 21:17 in 0.21666666666666667 minutes
completed chunk 440 queries at Feb 05, 2024 21:18 in 0.21666666666666667 minutes
completed chunk 450 queries at Feb 05, 2024 21:18 in 0.21666666666666667 minutes
completed chunk 460 queries at Feb 05, 2024 21:18 in 0.21666666666666667 minutes
completed chunk 470 queries at Feb 05, 2024 21:18 in 0.2 minutes
completed chunk 480 queries at Feb 05, 2024 

completed chunk 550 queries at Feb 05, 2024 21:20 in 0.2 minutes
completed chunk 560 queries at Feb 05, 2024 21:20 in 0.21666666666666667 minutes
completed chunk 570 queries at Feb 05, 2024 21:20 in 0.2 minutes
completed chunk 580 queries at Feb 05, 2024 21:20 in 0.18333333333333332 minutes
completed chunk 590 queries at Feb 05, 2024 21:21 in 0.18333333333333332 minutes
completed chunk 600 queries at Feb 05, 2024 21:21 in 0.2 minutes
completed chunk 610 queries at Feb 05, 2024 21:21 in 0.23333333333333334 minutes
completed chunk 620 queries at Feb 05, 2024 21:21 in 0.23333333333333334 minutes
completed chunk 630 queries at Feb 05, 2024 21:22 in 0.25 minutes
completed chunk 640 queries at Feb 05, 2024 21:22 in 0.25 minutes
completed chunk 650 queries at Feb 05, 2024 21:22 in 0.18333333333333332 minutes
completed chunk 660 queries at Feb 05, 2024 21:22 in 0.2 minutes
completed chunk 670 queries at Feb 05, 2024 21:22 in 0.21666666666666667 minutes
completed chunk 680 queries at Feb 05, 20

  published_time_df['published_time'] = pd.to_datetime(published_time_df['published_time'], utc=True)



...

completed chunk 1040 queries at Feb 05, 2024 21:30 in 0.2 minutes
completed chunk 1050 queries at Feb 05, 2024 21:30 in 0.21666666666666667 minutes
completed chunk 1060 queries at Feb 05, 2024 21:31 in 0.26666666666666666 minutes
completed chunk 1070 queries at Feb 05, 2024 21:31 in 0.18333333333333332 minutes
completed chunk 1080 queries at Feb 05, 2024 21:31 in 0.18333333333333332 minutes
completed chunk 1090 queries at Feb 05, 2024 21:31 in 0.25 minutes
completed chunk 1100 queries at Feb 05, 2024 21:32 in 0.2 minutes
completed chunk 1110 queries at Feb 05, 2024 21:32 in 0.25 minutes
completed chunk 1120 queries at Feb 05, 2024 21:32 in 0.23333333333333334 minutes
completed chunk 1130 queries at Feb 05, 2024 21:32 in 0.21666666666666667 minutes
completed chunk 1140 queries at Feb 05, 2024 21:32 in 0.18333333333333332 minutes
completed chunk 1150 queries at Feb 05, 2024 21:33 in 0.23333333333333334 minutes
completed chunk 1160 queries at Feb 05, 2024 21:33 in 0.21666666666666667 minu

completed chunk 1210 queries at Feb 05, 2024 21:34 in 0.21666666666666667 minutes
completed chunk 1220 queries at Feb 05, 2024 21:34 in 0.16666666666666666 minutes


  published_time_df['published_time'] = pd.to_datetime(published_time_df['published_time'], utc=True)

  published_time_df['published_time'] = pd.to_datetime(published_time_df['published_time'], utc=True)



completed chunk 1230 queries at Feb 05, 2024 21:35 in 0.23333333333333334 minutes


ConnectionError: HTTPSConnectionPool(host='search-finance-content-search-53kxgsezmqf7cyna2ccm67hipi.ap-south-1.es.amazonaws.com', port=443): Max retries exceeded with url: /finance-content/_search (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x168c69710>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

In [29]:
old_i = i

In [30]:
len(search_result_articles)

1231

In [31]:
for i in range(i, num_chunks):
    query_chunk = queries_df.iloc[i*chunk_size:(i+1)*chunk_size]
    start_time = time.time()
    with ThreadPoolExecutor(max_workers=30) as executor:
        futures = [executor.submit(get_top_hits_for_query_row, row) for index, row in query_chunk.iterrows()]
        for future in as_completed(futures):
            response = future.result()
            if response:  # Timeout or other errors
                search_result_articles.append(response)
    print(f'completed chunk {(i + 1) * chunk_size} queries at {"{:%b %d, %Y %H:%M}".format(datetime.now())} in {int(time.time() - start_time)/60} minutes')

completed chunk 1240 queries at Feb 05, 2024 21:42 in 0.26666666666666666 minutes
completed chunk 1250 queries at Feb 05, 2024 21:42 in 0.18333333333333332 minutes


completed chunk 1260 queries at Feb 05, 2024 21:42 in 0.2 minutes
completed chunk 1270 queries at Feb 05, 2024 21:42 in 0.26666666666666666 minutes
completed chunk 1280 queries at Feb 05, 2024 21:43 in 0.25 minutes
completed chunk 1290 queries at Feb 05, 2024 21:43 in 0.3 minutes
completed chunk 1300 queries at Feb 05, 2024 21:43 in 0.2833333333333333 minutes
completed chunk 1310 queries at Feb 05, 2024 21:43 in 0.23333333333333334 minutes
completed chunk 1320 queries at Feb 05, 2024 21:44 in 0.25 minutes
completed chunk 1330 queries at Feb 05, 2024 21:44 in 0.21666666666666667 minutes
completed chunk 1340 queries at Feb 05, 2024 21:44 in 0.23333333333333334 minutes
completed chunk 1350 queries at Feb 05, 2024 21:44 in 0.26666666666666666 minutes
completed chunk 1360 queries at Feb 05, 2024 21:45 in 0.2 minutes
completed chunk 1370 queries at Feb 05, 2024 21:45 in 0.3 minutes
completed chunk 1380 queries at Feb 05, 2024 21:45 in 0.23333333333333334 minutes
completed chunk 1390 queries 

completed chunk 1440 queries at Feb 05, 2024 21:47 in 0.21666666666666667 minutes


completed chunk 1450 queries at Feb 05, 2024 21:47 in 0.2 minutes
completed chunk 1460 queries at Feb 05, 2024 21:47 in 0.18333333333333332 minutes


  published_time_df['published_time'] = pd.to_datetime(published_time_df['published_time'], utc=True)



...

completed chunk 2430 queries at Feb 05, 2024 22:08 in 0.23333333333333334 minutes
completed chunk 2440 queries at Feb 05, 2024 22:08 in 0.23333333333333334 minutes
completed chunk 2450 queries at Feb 05, 2024 22:08 in 0.2 minutes
completed chunk 2460 queries at Feb 05, 2024 22:09 in 0.18333333333333332 minutes
completed chunk 2470 queries at Feb 05, 2024 22:09 in 0.18333333333333332 minutes
completed chunk 2480 queries at Feb 05, 2024 22:09 in 0.18333333333333332 minutes


completed chunk 2490 queries at Feb 05, 2024 22:09 in 0.18333333333333332 minutes
completed chunk 2500 queries at Feb 05, 2024 22:09 in 0.2 minutes
completed chunk 2510 queries at Feb 05, 2024 22:10 in 0.2833333333333333 minutes
completed chunk 2520 queries at Feb 05, 2024 22:10 in 0.2 minutes
completed chunk 2530 queries at Feb 05, 2024 22:10 in 0.2 minutes
completed chunk 2540 queries at Feb 05, 2024 22:10 in 0.2 minutes


completed chunk 2550 queries at Feb 05, 2024 22:11 in 0.21666666666666667 minutes
completed chunk 2560 queries at Feb 05, 2024 22:11 in 0.2 minutes
completed chunk 2570 queries at Feb 05, 2024 22:11 in 0.18333333333333332 minutes
completed chunk 2580 queries at Feb 05, 2024 22:11 in 0.26666666666666666 minutes
completed chunk 2590 queries at Feb 05, 2024 22:12 in 0.21666666666666667 minutes
completed chunk 2600 queries at Feb 05, 2024 22:12 in 0.23333333333333334 minutes


completed chunk 2610 queries at Feb 05, 2024 22:12 in 0.13333333333333333 minutes
completed chunk 2620 queries at Feb 05, 2024 22:12 in 0.21666666666666667 minutes
completed chunk 2630 queries at Feb 05, 2024 22:12 in 0.18333333333333332 minutes
completed chunk 2640 queries at Feb 05, 2024 22:13 in 0.31666666666666665 minutes
completed chunk 2650 queries at Feb 05, 2024 22:13 in 0.21666666666666667 minutes


completed chunk 2660 queries at Feb 05, 2024 22:13 in 0.36666666666666664 minutes
completed chunk 2670 queries at Feb 05, 2024 22:13 in 0.21666666666666667 minutes
completed chunk 2680 queries at Feb 05, 2024 22:14 in 0.25 minutes
completed chunk 2690 queries at Feb 05, 2024 22:14 in 0.26666666666666666 minutes
completed chunk 2700 queries at Feb 05, 2024 22:14 in 0.26666666666666666 minutes
completed chunk 2710 queries at Feb 05, 2024 22:15 in 0.25 minutes
completed chunk 2720 queries at Feb 05, 2024 22:15 in 0.25 minutes
completed chunk 2730 queries at Feb 05, 2024 22:15 in 0.2 minutes


In [ ]:
pd.DataFrame(search_result_articles).to_csv('search_results_corrected_5_feb.csv',index=False)

fetched results from cohere in 0:00:02.567009 second
fetched results from cohere in 0:00:02.714627 second
fetched results from cohere in 0:00:02.853046 second
fetched results from cohere in 0:00:03.279097 second
fetched results from cohere in 0:00:05.367942 second


In [150]:
pd.DataFrame(search_result_articles).to_csv('search_results_5_feb.csv',index=False)

In [77]:
from copy import deepcopy

In [86]:
df = deepcopy(queries_df)

In [87]:
pd.merge(df, pd.DataFrame(search_result_articles), how = 'left', left_on = ['query', 'query_date'], right_on = [1,2])

KeyError: 1

In [75]:
search_result_articles

[(2,
  'gold investment',
  '01/01/24',
  '["65bb096df3aa2a77a58bfb1a", "65ba2849f3aa2a77a58bf7dc", "65ba284cf3aa2a77a58bf800", "65bb4189f3aa2a77a58bfc43", "65ba2850f3aa2a77a58bf82f", "65bde481f3aa2a77a58c052b", "65bb41a1f3aa2a77a58bfd4e", "65b8d6ce6f6812560effade4", "65b89e891b79671c6b70d86d", "65b8d7566f6812560effae0b"]'),
 (1,
  'security token offerings news',
  '05/02/24',
  '["65c0bfcbc119ff6e57a3f11a", "65a91cba6391ac19baa826ec", "65a91ec96391ac19baa84336", "65a91b766391ac19baa81636", "65a920af6391ac19baa85d92", "65bf3601c119ff6e57a3ed78", "65a91c776391ac19baa8235e", "656e925c9994965ac4b4d6b4", "65a91d4a6391ac19baa82ea0", "65bccb50f3aa2a77a58c035f"]'),
 (4,
  'social media influencers',
  '16/02/24',
  '["65a7c2a01c9bc64bb8ae4a12", "65a7b7a01c9bc64bb8ae2056", "65a7b6f11c9bc64bb8ae1731", "658ff8404b25ce4814fb4113", "65a91f2c6391ac19baa8489c", "65a9200e6391ac19baa854d6", "65a77b083c864051fa47eede", "65a91e176391ac19baa839ae", "65a91c566391ac19baa821a2", "6555cc0e4b13023f934a475d"]

In [ ]:
def format_as_prompt_response(response_row):
    response = response_row['13']
    input_prompt = response_row['12']
    sys_prompt, user_content = input_prompt.split('|query_json|')
    messages = [{"role": "system", "content": sys_prompt},
                {"role": "user", "content": user_content}]
    context_prompt = tokenizer.decode(tokenizer.apply_chat_template(messages, add_generation_prompt=True))
    prompt = context_prompt + response
    prompt = re.sub(r'\n+','\n',prompt)
    return prompt

In [7]:
len(tokenizer.encode(system_prompt))

569